The master_file serves as the glue for all other files in the project.  It is the central file to load training and test data, tag the addresses, standardize the addresses, and compare the different address lists.

In [12]:
from collections import defaultdict
from address_compare import reference_data as refdt
from address_compare import standardizers as stndrdzr
from address_compare import comparers as comps
from address_compare import matcher as mtch
from address_compare.crf_tagger import AddressTagger
import json
import pandas as pd

In [13]:
retrain_crf_tagger = False

In [14]:
# Placeholder for reading/calling the training data for the CRF Tagger and sending the training data to train the model
if retrain_crf_tagger:
    with open('data/tagged_addresses.json') as f:
        td = json.load(f)
    
    #send training data to CRF tagger to train the model here...

In [15]:
# Placeholder for reading/calling the 2 lists of raw addresses
raw_address_list_1 = None
raw_address_list_2 = None

In [16]:
# instantiate AddressTagger object with default options, which gives the model trained in `Train CRF Model`.ipynb
at = AddressTagger()

In [17]:
# Placeholder for calling the trained CRF Tagger on the 2 lists of raw addresses
tagged_address_list_1 = at.series_to_address_df(raw_address_list_1["address"], standardize = True)
tagged_address_list_2 = at.series_to_address_df(raw_address_list_2["address"], standardize = True)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Intra-Grouping of Tagged Address Lists to Consolidate Duplicates
grouped_address_list_1 = stndrdzr.consolidate_address_list(tagged_address_list_1)
grouped_address_list_2 = stndrdzr.consolidate_address_list(tagged_address_list_2)